In [ ]:
#@title Instantiating an environment{vertical-output: true}
import car
import numpy as np
import PIL.Image
# from NavTask import Navigation
from ButtonTask import PressWithSpecificForce
from dm_control import viewer
from dm_control import composer
import numpy as np

creature = car.Car()
task = PressWithSpecificForce(creature)
# task = CarTask()
env = composer.Environment(task, random_state=np.random.RandomState(1))

env.reset()
PIL.Image.fromarray(env.physics.render())

In [ ]:
# from ButtonTask import PressWithSpecificForce
# from NavTask import Navigation
from SurvivalTask import Survive
# from MazeTask import Maze
from dm_control import viewer
from dm_control import composer
import numpy as np
from car import Car
from Point_Cloud_Env import CarEnv

creature = Car()

task = Survive(creature,150,9)
# task = Maze(creature)
def make_env():
    return CarEnv()

env = make_env()
# original_env = env.original_env
# task = PressWithSpecificForce(creature)
original_env = composer.Environment(task, raise_exception_on_physics_error=False, strip_singleton_obs_buffer_dim=True)

print(original_env.physics.data.ncon)
def random_policy(time_step):
# del time_step # Unused.
# return np.random.uniform(low=np.array([-0.38, -1.]), high=np.array([0.38, 3.]), size=(2,))
    return np.array([0 , 1])
viewer.launch(original_env, policy=random_policy)

# task.initialize_episode(original_env.physics,None)
# viewer.launch(original_env, policy=random_policy)

# collisions = task.detect_collisions(original_env.physics)
# print(collisions)

In [ ]:
from Point_Cloud_Env import CarEnv

# env = CarEnv()  # or Point_Cloud_Env()
# obs, _ = env.reset()
# for _ in range(100):
#     action = env.action_space.sample()
#     obs, reward, done, truncated, info = env.step(action)
#     if done or truncated:
#         obs, _ = env.reset()
# print("Single environment runs smoothly.")

for i in range(10):
    print(f"Testing initialization {i}")
    env = CarEnv()
    obs, _ = env.reset()
    env.close()
print("Repeated environment creation succeeded.")


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
# from ButtonEnv import CarEnv
# from NavEnv import CarEnv
# from ImageEnv import CarEnv
from Point_Cloud_Env import CarEnv
from GNN_Arch import point_cloud_to_graph, count_neighbors
import torch
from scipy.spatial import distance_matrix
from GNN_Arch import visualize_graph

np.set_printoptions(threshold=np.inf)

def make_env():
    return CarEnv()

env = make_env()
env.reset()

# action = np.array([0.2,0])
# env.step(action)
obs = env.getCurrObs()
point_cloud = obs['pc']

# print(point_cloud)

# Compute pairwise distances
# dist_matrix = distance_matrix(point_cloud, point_cloud)

# # Exclude self-distances (set diagonal to a high value)
# np.fill_diagonal(dist_matrix, np.inf)

# # Find the closest non-self distances for each point
# min_distances = np.min(dist_matrix, axis=1)

# # Determine an appropriate radius as a percentile of these distances
# recommended_radius = np.percentile(min_distances, 95)  # Using the 95th percentile

# # Determine max_num_neighbors based on typical connectivity
# max_num_neighbors = int(np.percentile(np.sum(dist_matrix < recommended_radius, axis=1), 95))

# print(recommended_radius, max_num_neighbors)

# np.save("point_cloud.npy", point_cloud)  # Save as NumPy array

# mean = point_cloud.mean(axis=0,keepdims=True)
# std = point_cloud.std(axis=0,keepdims=True) + 1e-6
# point_cloud = (point_cloud - mean) / std
# print('maximum x:',np.max(point_cloud[:,0]), 'maximum y:', np.max(point_cloud[:,1]))
# print(np.min(obs['pc'][:,2]),np.max(obs['pc'][:,2]))

edge_index, vertex_features, batch = point_cloud_to_graph(torch.tensor(obs['pc']).unsqueeze(0), method='knn', k = 8, radius=2)
print('max x:',vertex_features[:,0].max(), 'max y:',vertex_features[:,1].max(), 'max z:',vertex_features[:,2].max())
print('min x:',vertex_features[:,0].min(), 'min y:',vertex_features[:,1].min(), 'min z:',vertex_features[:,2].min())
visualize_graph(point_cloud,edge_index)
# count = count_neighbors(point_graph.edge_index,64*128)
# print(count,torch.sum(count==max(count)),min(count))

In [ ]:
import numpy as np

default_fovy_degrees = 45.0
default_fovy_radians = np.deg2rad(default_fovy_degrees)
height = 64
width = 128
aspect_ratio = 1  # Explicitly calculate aspect ratio

fy_calculated = height / (2 * np.tan(default_fovy_radians / 2))
fx_calculated = fy_calculated * aspect_ratio
cx_calculated = width / 2
cy_calculated = height / 2

print("Calculated Camera Intrinsics (based on default fovy=45 and aspect=width/height):")
print(f"fx: {fx_calculated:.2f}")
print(f"fy: {fy_calculated:.2f}")
print(f"cx: {cx_calculated:.2f}")
print(f"cy: {cy_calculated:.2f}")

print("\nOriginal Hardcoded Intrinsics (from car.py):")
print(f"fx_hardcoded: 382.57")
print(f"fy_hardcoded: 382.57")
print(f"cx_hardcoded: {width / 2}")
print(f"cy_hardcoded: {height / 2}")

In [1]:
import torch

import os
import argparse
import time

from stable_baselines3 import SAC, PPO
# from sbx import SAC

from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
import torch.optim as optim

print(torch.__version__)
torch.cuda.is_available()

2.6.0+cu124


/home/mmkr/LocoTransformer_v2/NewEnv/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [ ]:
# from NavEnv import CarEnv
# from ButtonEnv import CarEnv
# from SurvivalEnv import CarEnv
from ImageEnv import CarEnv
# from Point_Cloud_Env import CarEnv
from feature_extractors import PointGNNFeatureExtractorWrapper, CNNFeatureExtractor
import os
from torch.cuda.amp import autocast

# os.environ["MJ_THREADS"] = "4"


def make_env(rank):
    def _init():
        # print(f"Initializing environment {rank} in process {os.getpid()}")
        return CarEnv()
    return _init

tb = os.path.join("/home/mmkr/LocoTransformer_v2/Pushr_car_simulation/results/tensor","SAC_2DCNN_300k")
models = os.path.join("/home/mmkr/LocoTransformer_v2/Pushr_car_simulation/results/models","SAC_2DCNN_300k")
stats_path = os.path.join("/home/mmkr/LocoTransformer_v2/Pushr_car_simulation/results/logs","SAC_2DCNN_300k")

num_envs = 8
env = SubprocVecEnv([make_env(i) for i in range(num_envs)])
# env = DummyVecEnv([make_env(i) for i in range(num_envs)])
env = VecNormalize(env, norm_obs=False, norm_reward=True)


# policy_kwargs = dict(
#     features_extractor_class=PointGNNFeatureExtractorWrapper,
#     features_extractor_kwargs=dict(input_dim=3,hidden_dim=32,output_dim=256,num_layers=3,use_edgeconv=False),
#     net_arch=[256,256]
# )

policy_kwargs = dict(
    features_extractor_class=CNNFeatureExtractor,
    features_extractor_kwargs=dict(features_dim=128),
    net_arch=[256,256]
)


# model = PPO(
#     policy='MultiInputPolicy',
#     n_steps = 2048,
#     learning_rate=3e-4,
#     n_epochs=10,
#     env=env,
#     batch_size=16,
#     clip_range=0.2,
#     ent_coef=0.01,
#     vf_coef=0.5,
#     verbose=1,
#     tensorboard_log=tb,
#     seed=1,
#     policy_kwargs=policy_kwargs
# )


model = SAC(
    policy='MultiInputPolicy',
    env=env,
    learning_rate=3e-4,
    batch_size=128,
    buffer_size=150000,
    learning_starts=1000,
    tensorboard_log=tb,
    policy_kwargs=policy_kwargs,
    verbose=1
)


# saved_model_path='/home/mmkr/LocoTransformer_v2/Pushr_car_simulation/results/models/PPO_GS_100k_32_128'
# model=PPO.load(saved_model_path, env=env)
# model.policy.half()

# with torch.amp.autocast(device_type="cuda"):
model.learn(total_timesteps=300000)
env.save(stats_path+'test'+".pkl")
print("Saving end model")
model.save(models)

# model.policy

In [2]:
import os
import argparse
import time

from stable_baselines3 import SAC, PPO
# from sbx import SAC

from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
import torch.optim as optim

import car
import numpy as np
from Point_Cloud_Env import CarEnv
# from ImageEnv import CarEnv
from SurvivalTask import Survive
from dm_control import viewer,composer

env=CarEnv()

task = env.task
# task = Maze(creature)
# task = PressWithSpecificForce(creature)
original_env = env.original_env

saved_model_path='results/models/SAC_GC_700k_4N_32_256.zip'
model=SAC.load(saved_model_path, env=env, custom_objects={'buffer_size': 1})
# model.replay_buffer = None
# model.set_env(env=env)
state,_ = env.reset()
# # model.policy

def policy(time_step,model):
    
    pc = time_step.observation['car/compute_point_cloud']
    obs = {"pc": pc}
    # vec = time_step.observation['car/body_vel_2d']
    # img = time_step.observation['car/realsense_camera']
    # obs = {"vec":vec, "img":img}

    action, _=model.predict(obs,deterministic=True)
    # obs, reward, done, _,_ = env.step(action)
    # print(action)
    
    return action

viewer.launch(original_env, policy=lambda timestep: policy(timestep, model))

/home/mmkr/LocoTransformer_v2/NewEnv/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Dist 0


In [ ]:
import os
import torch
import argparse
from stable_baselines3 import SAC, PPO
# from sb3_contrib import RecurrentPPO
ext_list = ["pi_features_extractor", "mlp_extractor.policy_net", "action_net"]
"""
pi_features_extractor:
    Input: Depth image format -> (1,128,128,1)
    output: 128 features ---- check for the shape of the output
mlp_extractor.policy_net:
    input: pi_features_extractor_output --> 128
    output: 128 ----> check for the shape of the output
action_net:
    input: mlp_extractor.policy_net --> 128
    output: 2 ----> check for the shape of the output
"""
class DictInputWrapper(torch.nn.Module):
    def __init__(self, policy):
        super(DictInputWrapper, self).__init__()
        self.policy = policy
    def forward(self, depth, vec):
        # Recreate the observation dictionary
        obs = {'vec': vec, 'img': depth}
        # Forward pass through the actual policy using dict observation
        return self.policy(obs)
    
    
def export_to_onnx(model_part, input_tensors, output_path, input_names=None, output_names=None):
    # Use the provided input names or default to "input1", "input2", etc.
    if input_names is None:
        input_names = [f"input_{i}" for i in range(len(input_tensors))]
    if output_names is None:
        output_names = ["output"]

    # Prepare dynamic axes for each input
    dynamic_axes = {name: {0: 'batch_size'} for name in input_names}

    torch.onnx.export(
        model_part,
        input_tensors,
        output_path,
        export_params=True,
        opset_version=11,
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axes
    )
    print(f"Exported {output_path} to ONNX successfully!")


def save_model_parts_as_onnx(model, output_dir="onnx_models"):
    os.makedirs(output_dir, exist_ok=True)
    model.policy.eval()

    # Determine the device (CPU or GPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Extract CNN Network (if it only processes 'img')
    print("Extracting and converting CNN Network...")
    dummy_img = torch.randn(1, *model.observation_space['img'].shape).to(device)
    cnn_wrapper = DictInputWrapper(model.policy.features_extractor)
    export_to_onnx(
        cnn_wrapper,
        (torch.zeros_like(dummy_img), dummy_img),  # vec is zeros
        os.path.join(output_dir, "CNN.onnx"),
        input_names=["vec", "img"],
        output_names=["output"]
    )

    # Extract MLP Network (if it processes combined features)
    print("Extracting and converting MLP Network...")
    # Determine the input size for the MLP; this depends on your model architecture
    # For example, if the MLP processes features of size N, create a dummy input of that size
    dummy_features = torch.randn(1, model.policy.mlp_extractor.policy_net[0].in_features).to(device)
    export_to_onnx(
        model.policy.mlp_extractor.policy_net,
        dummy_features,
        os.path.join(output_dir, "MLP.onnx"),
        input_names=["input"],
        output_names=["output"]
    )

    # Extract Action Network
    print("Extracting and converting Action Network...")
    export_to_onnx(
        model.policy.action_net,
        dummy_features,
        os.path.join(output_dir, "ActionNet.onnx"),
        input_names=["input"],
        output_names=["output"]
    )


def load_model_and_save_parts(model_type, model_path, output_dir):
    # model_class = {"PPO": PPO, "SAC": SAC}[model_type]
    # Load the model from .zip file to ensure weights are correct
    model = PPO.load(model_path)
    print("Model loaded successfully!")
    # Save parts to ONNX
    save_model_parts_as_onnx(model, output_dir)


if __name__ == "__main__":
    # parser = argparse.ArgumentParser(description="Extract and convert parts of a saved model to ONNX format.")
    # parser.add_argument('--model_type', type=str, choices=["PPO", "SAC", "RecurrentPPO"], help='Type of model to use', default="SAC")
    # parser.add_argument('--model_path', type=str, help='Path to the saved model (.zip) to extract parts from', required=True)
    # parser.add_argument('--output_dir', type=str, help='Directory to save the ONNX files', default='./onnx_models')
    # args = parser.parse_args()
    model_type = PPO
    model_path = "/home/mmkr/Car_LocoTransformer/Mujoco_learning/Pushr_car_simulation/results/models/PPO_Img_Vel_300k"
    output_dir = "/home/mmkr/Car_LocoTransformer/Mujoco_learning/Pushr_car_simulation/results/onnx"
    # Execute the extraction and conversion
    # load_model_and_save_parts(args.model_type, args.model_path, args.output_dir)
    load_model_and_save_parts(model_type, model_path, output_dir)